# Import necessary packages

In [5]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
import  preprocess as prep


# Load the data


In [6]:
fraud_data = pd.read_csv('../data/raw/Fraud_Data.csv')
ip_data = pd.read_csv('../data/raw/IpAddress_to_Country.csv')

# Handle missing values


In [7]:
fraud_data = prep.handle_missing_values(fraud_data)

# Data cleaning


In [8]:
fraud_data = prep.data_cleaning(fraud_data)